In [1]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
from steam import WebAPI
from datetime import datetime
import os
import glob
import json
import tensorflow as tf
import sys
from IPython import display
import requests
import tensorflow.keras.backend as K
from sklearn import decomposition

#import logging
#logging.getLogger("tensorflow").setLevel(logging.ERROR)

#from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()
tf.autograph.set_verbosity(0)
print(tf.__version__)

2.1.0


In [2]:
sns.set_style('ticks')

In [3]:
champfile=os.path.join('lol_data','version9.10.1_champion.json')
with open(champfile,'r') as fin:
    champdata=json.load(fin)
champ_vocab=[]
for name,data in champdata['data'].items():
    champ_vocab.append(data['key'])
hero_vocab = [int(i) for i in champ_vocab]
vocab_len=len(hero_vocab)
print(len(hero_vocab))

144


# Build TFRecord input pipeline

## Parse function

In [4]:
def _parse_function_extra_info(example_proto):
    """Extracts features and labels.
  
    Args:
        example_proto: tf.Example protocol (unsure what this is)    
      Returns:
    A `tuple` `(labels, features)`:
      features: A dict of tensors representing the features
      labels: A tensor with the corresponding labels.
    """
    features = {}
    #there are 5 heroes per team 
    heroes_per_team=5
    #there are 4 pieces of info for a hero in LOL
    num_info_fields=4
    num_tags=6 #6 tags
    num_valid_stats=14 #14 valid stats (after removing mana and crit)
    #loop through five times to capture each hero on each team
    for i in range(heroes_per_team):
        #64 bit int for hero
        features["radiant_hero"+str(i)]=tf.io.FixedLenFeature(1, dtype=tf.int64)

        features["radiant_hero_info"+str(i)]=tf.io.FixedLenFeature(num_info_fields,
                                                                   dtype=tf.float32) 
        features["radiant_hero_tags"+str(i)] =tf.io.FixedLenFeature(num_tags,
                                                                    dtype=tf.int64) 
        features["radiant_hero_stats"+str(i)] =tf.io.FixedLenFeature(num_valid_stats,
                                                                     dtype=tf.float32) 
        
        features["dire_hero"+str(i)] =tf.io.FixedLenFeature(1, dtype=tf.int64) 
        features["dire_hero_info"+str(i)] =tf.io.FixedLenFeature(num_info_fields,
                                                                 dtype=tf.float32) 
        features["dire_hero_tags"+str(i)] =tf.io.FixedLenFeature(num_tags,
                                                                 dtype=tf.int64) 
        features["dire_hero_stats"+str(i)] =tf.io.FixedLenFeature(num_valid_stats,
                                                                  dtype=tf.float32) 
 
    #there are 11 different regions (at least in my most current dataset)
    num_regions=11
    #two skill levels in the lol api
    num_skill_levels=2
    #On patch for current data (9.10.1) there are 144 characters
    num_heroes=144

    features["region"] = tf.io.FixedLenFeature(num_regions, dtype=tf.int64)
    features["skill_level"] = tf.io.FixedLenFeature(1, dtype=tf.int64)
    
    features["target"] = tf.io.FixedLenFeature(1, dtype=tf.int64)
    
    #parse features
    parsed_features = tf.io.parse_single_example(example_proto, features)
    
    #load features into feature dict
    feature_dict={}
    #stack hero input so that each hero is treated the same in model
    feature_dict['radiant_heroes']=tf.stack([
                    tf.one_hot(parsed_features['radiant_hero0'],num_heroes,axis=0),
                    tf.one_hot(parsed_features['radiant_hero1'],num_heroes,axis=0),
                    tf.one_hot(parsed_features['radiant_hero2'],num_heroes,axis=0),
                    tf.one_hot(parsed_features['radiant_hero3'],num_heroes,axis=0),
                    tf.one_hot(parsed_features['radiant_hero4'],num_heroes,axis=0)])
    feature_dict['radiant_hero_info']=tf.stack([
                    parsed_features['radiant_hero_info0'],
                    parsed_features['radiant_hero_info1'],
                    parsed_features['radiant_hero_info2'],
                    parsed_features['radiant_hero_info3'],
                    parsed_features['radiant_hero_info4']])
    feature_dict['radiant_hero_tags']=tf.stack([
                    parsed_features['radiant_hero_tags0'],
                    parsed_features['radiant_hero_tags1'],
                    parsed_features['radiant_hero_tags2'],
                    parsed_features['radiant_hero_tags3'],
                    parsed_features['radiant_hero_tags4']])
    feature_dict['radiant_hero_stats']=tf.stack([
                    parsed_features['radiant_hero_stats0'],
                    parsed_features['radiant_hero_stats1'],
                    parsed_features['radiant_hero_stats2'],
                    parsed_features['radiant_hero_stats3'],
                    parsed_features['radiant_hero_stats4']])
    feature_dict['dire_heroes']=tf.stack([
                    tf.one_hot(parsed_features['dire_hero0'],num_heroes,axis=0),
                    tf.one_hot(parsed_features['dire_hero1'],num_heroes,axis=0),
                    tf.one_hot(parsed_features['dire_hero2'],num_heroes,axis=0),
                    tf.one_hot(parsed_features['dire_hero3'],num_heroes,axis=0),
                    tf.one_hot(parsed_features['dire_hero4'],num_heroes,axis=0)])
    feature_dict['dire_hero_info']=tf.stack([
                    parsed_features['dire_hero_info0'],
                    parsed_features['dire_hero_info1'],
                    parsed_features['dire_hero_info2'],
                    parsed_features['dire_hero_info3'],
                    parsed_features['dire_hero_info4']])
    feature_dict['dire_hero_tags']=tf.stack([
                    parsed_features['dire_hero_tags0'],
                    parsed_features['dire_hero_tags1'],
                    parsed_features['dire_hero_tags2'],
                    parsed_features['dire_hero_tags3'],
                    parsed_features['dire_hero_tags4']])
    feature_dict['dire_hero_stats']=tf.stack([
                    parsed_features['dire_hero_stats0'],
                    parsed_features['dire_hero_stats1'],
                    parsed_features['dire_hero_stats2'],
                    parsed_features['dire_hero_stats3'],
                    parsed_features['dire_hero_stats4']])
    feature_dict['skill_level']= tf.one_hot(parsed_features['skill_level'],
                                            num_skill_levels)
    feature_dict['region']=parsed_features['region']
    target = parsed_features['targets']
    
    return feature_dict, target

#### Check the parse function worked

In [5]:
# Create the Dataset object.
ds = tf.data.TFRecordDataset(os.path.join('LOL_data',
                                          'mix_tier',
                                          'lol_training_data',
                                          'blue_win_extra_info_dota2_compat_lol_training_data.tfrecords'),
                                          compression_type="GZIP")
# Map features and labels with the parse function.
ds = ds.map(_parse_function_extra_info)

In [6]:
ds = ds.shuffle(10000)
print(next(iter(ds)))

({'radiant_heroes': <tf.Tensor: shape=(5, 144, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
   

## Input functions

In [7]:
# Create an input_fn that parses the tf.Examples from the given files,
# and split them into features and targets.
def _input_fn_extra_info(input_filenames, num_epochs=None, 
              shuffle=True, batch_size=50,compression_type=""):
   
   # Same code as above; create a dataset and map features and labels.
    ds = tf.data.TFRecordDataset(input_filenames,compression_type=compression_type)
    ds = ds.map(_parse_function_extra_info)

    if shuffle:
        ds = ds.shuffle(10000)
    ds = ds.batch(batch_size)
    ds = ds.repeat()
    
    # Return the dataset.
    return ds

## define feature column constructor functions

In [8]:
def construct_numeric_columns(keys,shapes):
    """Construct categorical features for all features
        Args: 
            input_features: names of input feature columns to use
        returns:
            a set of categorical columns
    """
    return [tf.feature_column.numeric_column(key,shape=shape)
                                    for key,shape in zip(keys,shapes)]                                          

# Train the model

## define model training function
also includes saving and logging

In [33]:
def train_model(
        optimizer,
        model_type,
        steps,
        batch_size,
        feature_columns,
        training_file,
        validation_file,
        compression_type='',
        temp_log_flag=True,
        hidden_units=[],
        dnn_feature_columns=None):
    """Trains a classification model.
  
    In addition to training, this function also prints training progress information,
    as well as a plot of the training and validation loss over time.
      
    Args:
        optimizer: A `tf.Optimizer`, the optimizer to use for gradient descent
        steps: A non-zero `int`, the total number of training steps. A training step
          consists of a forward and backward pass using a single batch.
        batch_size: A `int` specifying the number of examples for each training step.
        feature_columns: A `set` specifying the input feature columns to use.
        training_file: A `string`  pointing to the training file.
        validation_file: A `string` pointing to the validation file.
        compression_type: A `string` specifying compression type
        temp_log_flag: A `bool` specifying whether to save logs in temp folder
        
    Returns:
        A `Classifier` object trained on the training data.
    """

    #allow logging in temp directory or directory to be included in git
    #  useful if tuning hyperparams or testing
    if temp_log_flag:
        logdir = os.path.join('..','..','..','tmp_log_dir_lol',
                          datetime.now().strftime("lol_%Y%m%d-%H%M%S"))
    else:
        logdir = os.path.join('log_dir',
                          datetime.now().strftime("%Y%m%d-%H%M%S"))
    
    #set config options
    config=tf.estimator.RunConfig(model_dir=logdir,
                                  save_summary_steps=20)
    # Create a classifier object.
    if model_type=='linear':
        classifier = tf.estimator.LinearClassifier(
              feature_columns=feature_columns,
              optimizer=optimizer,
              config=config)
    elif model_type=='dnn':
        classifier = tf.estimator.DNNClassifier(
              hidden_units=hidden_units,
              feature_columns=feature_columns,
              optimizer=optimizer,
              config=config)
    elif model_type=='dnn_linear':
        if dnn_feature_columns==None:
            dnn_feature_columns=feature_columns

        classifier = tf.estimator.DNNLinearCombinedClassifier(
              dnn_hidden_units=hidden_units,
              linear_feature_columns=feature_columns,
              dnn_feature_columns=dnn_feature_columns,
              linear_optimizer=optimizer,
              dnn_optimizer=optimizer,
              config=config)       
    else:
        raise KeyError('model_type not found. Got: '+ model_type)
        
    #define training and validation inputs
    training_input_fn =lambda: _input_fn_extra_info([training_file], 
                                         batch_size=batch_size,
                                         compression_type=compression_type)
    validation_input_fn =lambda: _input_fn_extra_info([validation_file], 
                                           batch_size=batch_size,
                                          compression_type=compression_type)

    # Train the model, but do so inside a loop so that we can periodically assess
    # loss metrics.
    print("Training model...")
    
    training_loss = []
    validation_loss = []
    # Train the model, starting from the prior state.
    classifier.train(input_fn=training_input_fn,
                                steps=steps)
        
    ## Compute predictions.
    evaluation_metrics = classifier.evaluate(
          input_fn=validation_input_fn,
          steps=steps)

    #save params for this training to a json in the logdir

    training_params = dict({'model_type':'single_target_'+model_type,
                            'steps':steps,
                            'batch_size':batch_size})
                            #'embedding_dims':embedding_dims,
                            #'optimizer':optimizer._name,
                            #'hyper_parameters':optimizer.get_config()})
    if len(hidden_units)>0:
        training_params['hidden_units']=hidden_units

    with open(os.path.join(logdir,'training_params.json'),'w') as fp:
        json.dump(training_params,fp)

    #save trained model
    #serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
    #                      tf.feature_column.make_parse_example_spec(feature_columns))
    #export_path = classifier.export_saved_model(log_dir, serving_input_fn)
    
    
    print("validation set metrics:")
    for m in evaluation_metrics:
        print(m, evaluation_metrics[m])
    print("---")
    
    return classifier


### Define training/validation files

In [27]:
training_file_novocab = os.path.join('LOL_data',
                             'mix_tier',
                             'lol_training_data',
                             'blue_win_extra_info_dota2_compat_lol_training_data.tfrecords')
validation_file_novocab = os.path.join('LOL_data',
                             'mix_tier',
                             'lol_validation_data',
                             'blue_win_extra_info_dota2_compat_lol_validation_data.tfrecords')

### define feature columns for all info, no_stats, no info/tags, or hero only

In [11]:
#shapes of each feature column  (five axis is for heroes per team)
hero_shape=(5,vocab_len);shape_info=(5,4);
shape_tags=(5,6);shape_stats=(5,14);
shape_skills=2;shape_region=11;

In [12]:
#construct categorical columns for each hero
feat_list_all = ['radiant_heroes','radiant_hero_info','radiant_hero_stats',
                 'radiant_hero_tags','dire_heroes','dire_hero_info',
                 'dire_hero_stats','dire_hero_tags']

feat_list_all.extend(['skill_level','region'])
feat_shape_list_all = [hero_shape,shape_info,shape_stats,shape_tags,
                   hero_shape,shape_info,shape_stats,shape_tags]
feat_shape_list_all.extend([shape_skills,shape_region])

feature_columns_all=construct_numeric_columns(feat_list_all,
                                              feat_shape_list_all)

In [13]:
feat_list_no_stats = ['radiant_heroes','radiant_hero_info','radiant_hero_tags',
                      'dire_heroes','dire_hero_info','dire_hero_tags']
feat_list_no_stats.extend(['skill_level','region'])
feat_shape_list_no_stats = [hero_shape,shape_info,shape_tags,
                           hero_shape,shape_info,shape_tags]
feat_shape_list_no_stats.extend([shape_skills,shape_region])
feature_columns_no_stats=construct_numeric_columns(feat_list_no_stats,
                                                   feat_shape_list_no_stats)

In [14]:
feat_list_no_info_tags = ['radiant_heroes','radiant_hero_stats',
                      'dire_heroes','dire_hero_stats']
feat_list_no_info_tags.extend(['skill_level','region'])
feat_shape_list_no_info_tags = [hero_shape,shape_stats,
                           hero_shape,shape_stats]
feat_shape_list_no_info_tags.extend([shape_skills,shape_region])
feature_columns_no_info_tags=construct_numeric_columns(feat_list_no_info_tags,
                                                   feat_shape_list_no_info_tags)

In [15]:
feat_list_hero_only = ['radiant_heroes','dire_heroes']
feat_list_hero_only.extend(['skill_level','region'])

feat_shape_list_hero_only = [hero_shape,hero_shape]
feat_shape_list_hero_only.extend([shape_skills,shape_region])

feature_columns_hero_only=construct_numeric_columns(feat_list_hero_only,
                                                   feat_shape_list_hero_only)

In [24]:
feat_list_hero_only_noregion = ['radiant_heroes','dire_heroes']
feat_list_hero_only_noregion.extend(['skill_level'])

feat_shape_list_hero_only_noregion = [hero_shape,hero_shape]
feat_shape_list_hero_only_noregion.extend([shape_skills])

feature_columns_hero_only_noregion=construct_numeric_columns(feat_list_hero_only_noregion,
                                                   feat_shape_list_hero_only_noregion)

### Train models

#### Linear test cases (for hero only, no_stats and full extra_info)

In [32]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
classifier = train_model(
            optimizer,
            model_type='linear',
            batch_size=1000, 
            steps=2000,
            feature_columns=feature_columns_all,
            training_file=training_file_novocab, 
            validation_file=validation_file_novocab,
            compression_type='GZIP')

INFO:tensorflow:Using config: {'_model_dir': '../../../tmp_log_dir_lol/lol_20200327-205411', '_tf_random_seed': None, '_save_summary_steps': 20, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Training model...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSave

In [34]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
classifier = train_model(
            optimizer,
            model_type='linear',
            batch_size=1000, 
            steps=2000,
            feature_columns=feature_columns_no_stats,
            training_file=training_file_novocab, 
            validation_file=validation_file_novocab,
            compression_type='GZIP')

INFO:tensorflow:Using config: {'_model_dir': '../../../tmp_log_dir_lol/lol_20200327-211517', '_tf_random_seed': None, '_save_summary_steps': 20, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Training model...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSave

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
classifier = train_model(
            optimizer,
            model_type='linear',
            batch_size=1000, 
            steps=2000,
            feature_columns=feature_columns_hero_only,
            training_file=training_file_novocab, 
            validation_file=validation_file_novocab,
            compression_type='GZIP')

INFO:tensorflow:Using config: {'_model_dir': '../../../tmp_log_dir_lol/lol_20200327-213241', '_tf_random_seed': None, '_save_summary_steps': 20, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Training model...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSave

#### DNN model testing

In [36]:
#p good accuracy of 55.8 (but not nearly as good as linear dnn)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
classifier = train_model(
            optimizer,
            model_type='dnn',
            batch_size=500, 
            steps=2000,
            feature_columns=feature_columns_all,
            training_file=training_file_novocab, 
            validation_file=validation_file_novocab,
            compression_type='GZIP',
            dropout=
            hidden_units=[4,4,2])

INFO:tensorflow:Using config: {'_model_dir': '../../../tmp_log_dir_lol/lol_20200327-215132', '_tf_random_seed': None, '_save_summary_steps': 20, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Training model...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSave

#### DNN + Linear (wide and deep) no_stats
this has the best performance of any model tested

In [41]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,amsgrad=True)
classifier = train_model(
            optimizer,
            model_type='dnn_linear',
            batch_size=5000, 
            steps=10000,
            feature_columns=feature_columns_no_stats,
            training_file=training_file_novocab, 
            validation_file=validation_file_novocab,
            compression_type='GZIP',
            hidden_units=[16,8,4,4,2])

INFO:tensorflow:Using config: {'_model_dir': '../../../tmp_log_dir_lol/lol_20200328-002446', '_tf_random_seed': None, '_save_summary_steps': 20, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Training model...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSave

INFO:tensorflow:global_step/sec: 1.52692
INFO:tensorflow:global_step/sec: 1.51662
INFO:tensorflow:loss = 0.65660274, step = 8800 (130.529 sec)
INFO:tensorflow:global_step/sec: 1.53093
INFO:tensorflow:global_step/sec: 1.52514
INFO:tensorflow:loss = 0.60219055, step = 9000 (131.654 sec)
INFO:tensorflow:Saving checkpoints for 9070 into ../../../tmp_log_dir_lol/lol_20200328-002446/model.ckpt.
INFO:tensorflow:global_step/sec: 1.52061
INFO:tensorflow:global_step/sec: 1.54008
INFO:tensorflow:loss = 0.6550729, step = 9200 (130.669 sec)
INFO:tensorflow:global_step/sec: 1.52765
INFO:tensorflow:global_step/sec: 1.52376
INFO:tensorflow:loss = 0.64141697, step = 9400 (130.811 sec)
INFO:tensorflow:global_step/sec: 1.52389
INFO:tensorflow:global_step/sec: 1.52003
INFO:tensorflow:loss = 0.6006461, step = 9600 (131.573 sec)
INFO:tensorflow:global_step/sec: 1.52557
INFO:tensorflow:global_step/sec: 1.53014
INFO:tensorflow:loss = 0.6611821, step = 9800 (130.955 sec)
INFO:tensorflow:global_step/sec: 1.5214

In [42]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,amsgrad=True)
classifier = train_model(
            optimizer,
            model_type='dnn_linear',
            batch_size=5000, 
            steps=15000,
            feature_columns=feature_columns_no_stats,
            training_file=training_file_novocab, 
            validation_file=validation_file_novocab,
            compression_type='GZIP',
            hidden_units=[16,8,4,4,2])

INFO:tensorflow:Using config: {'_model_dir': '../../../tmp_log_dir_lol/lol_20200328-055014', '_tf_random_seed': None, '_save_summary_steps': 20, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Training model...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSave

INFO:tensorflow:loss = 0.61372215, step = 8800 (108.494 sec)
INFO:tensorflow:global_step/sec: 1.83924
INFO:tensorflow:global_step/sec: 1.84303
INFO:tensorflow:loss = 0.6198256, step = 9000 (108.815 sec)
INFO:tensorflow:global_step/sec: 1.84677
INFO:tensorflow:global_step/sec: 1.83923
INFO:tensorflow:loss = 0.6262949, step = 9200 (108.530 sec)
INFO:tensorflow:global_step/sec: 1.84551
INFO:tensorflow:global_step/sec: 1.83542
INFO:tensorflow:loss = 0.6294072, step = 9400 (108.448 sec)
INFO:tensorflow:global_step/sec: 1.84441
INFO:tensorflow:Saving checkpoints for 9502 into ../../../tmp_log_dir_lol/lol_20200328-055014/model.ckpt.
INFO:tensorflow:global_step/sec: 1.8372
INFO:tensorflow:loss = 0.614905, step = 9600 (108.879 sec)
INFO:tensorflow:global_step/sec: 1.83349
INFO:tensorflow:global_step/sec: 1.84381
INFO:tensorflow:loss = 0.6109491, step = 9800 (108.763 sec)
INFO:tensorflow:global_step/sec: 1.8438
INFO:tensorflow:global_step/sec: 1.82884
INFO:tensorflow:loss = 0.6013543, step = 100

#### Testing wide and deep (dnn/linear) with other features

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,amsgrad=True)
classifier = train_model(
            optimizer,
            model_type='dnn_linear',
            batch_size=5000, 
            steps=5000,
            feature_columns=feature_columns_hero_only_noregion,
            training_file=training_file_novocab, 
            validation_file=validation_file_novocab,
            compression_type='GZIP',
            hidden_units=[32,32,16,8,4,2])